### 🧪 **Full Experiment Using the Library RADAR**

#### ⚙️ **Model**
> **Informer** — a Transformer-based architecture for efficient time series forecasting and anomaly detection.

#### 📊 **Dataset**
> **AI4I 2020 Predictive Maintenance**  
> Industrial sensor data used to predict and detect anomalies related to equipment degradation and failure.

#### 🎯 **Objective**
> **Anomaly detection in time series**  
> Identify abnormal behaviors and potential failures from multivariate sensor readings over time.

In [3]:
import sys
sys.path.append(".../RADAR_Plataform/RADAR") # dir library

In [15]:
from RADAR.time_series.time_series_utils import TimeSeriesProcessor
from RADAR.time_series.algorithms import tsfedl
from RADAR.time_series.algorithms import transformers
from RADAR.time_series.preprocessing.preprocessing_ts import StandardScalerPreprocessing, OneHotEncoderPreprocessing
from RADAR.time_series.time_series_datasets_uci import global_load as load_time_series 
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
from RADAR.visualization_module import DataVisualization,DataVisualizationScoresTS
import numpy as np

#### 📂 **Load Dataset**

The **training dataset** can be freely selected, along with an independent **test dataset**.  
Alternatively, you can use one of the **simple datasets from the [UCI Repository](https://archive.ics.uci.edu/)**, which can be loaded directly from the dataset module files:

- 📘 **Static data:** `static_datasets_uci`  
- ⏱️ **Time series data:** `time_series_datasets_uci`

For **federated models**, any of these dataset types can be used, depending on whether the model is designed for **static data** or **time series**.

In [12]:
X,y = load_time_series('ai4i_2020_predictive_maintenance_dataset')   #name dataset in static datasets uci repo
labels = y["Machine failure"] # choose target label or labels
print(y)


Metadata: {'uci_id': 601, 'name': 'AI4I 2020 Predictive Maintenance Dataset', 'repository_url': 'https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/601/data.csv', 'abstract': 'The AI4I 2020 Predictive Maintenance Dataset is a synthetic dataset that reflects real predictive maintenance data encountered in industry.', 'area': 'Computer Science', 'tasks': ['Classification', 'Regression', 'Causal-Discovery'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 10000, 'num_features': 6, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], 'index_col': ['UID', 'Product ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Wed Feb 14 2024', 'dataset_doi': '10.24432/C5HS5C', 'creators': [], 'intro_paper': {'ID': 386, 'type': 'NATIVE', 'title': 'Explainable Artificial 

#### 🧹 **Preprocessing Dataset**

It is possible to preprocess the dataset using one of the available functions in the **preprocessing** folder, corresponding to each data type within the library.  

In this example, we use:  
- **`StandardScalerPreprocessing`** — to normalize numerical features.  
- **`OneHotEncoderPreprocessing`** — to encode categorical features that require this type of preprocessing.
¿Quieres que la siguiente sección (“#### Train Model”) siga este mismo formato visual y tono técnico en  

In [ ]:
# X = X.drop('Type',axis=1)  # remove Type or apply one hot encoding 

In [19]:
encoder = OneHotEncoderPreprocessing(columns=['Type'])
X_encoded = encoder.fit_transform(X)

print("Data after encoding:")
print(X_encoded)

# X_decoded = encoder.inverse_transform(X_encoded)
# print("\nData after reversal:")
# print(X_decoded)

Data after encoding:
      Air temperature  Process temperature  Rotational speed  Torque  \
0               298.1                308.6              1551    42.8   
1               298.2                308.7              1408    46.3   
2               298.1                308.5              1498    49.4   
3               298.2                308.6              1433    39.5   
4               298.2                308.7              1408    40.0   
...               ...                  ...               ...     ...   
9995            298.8                308.4              1604    29.5   
9996            298.9                308.4              1632    31.8   
9997            299.0                308.6              1645    33.4   
9998            299.0                308.7              1408    48.5   
9999            299.0                308.7              1500    40.2   

      Tool wear  Type_H  Type_L  Type_M  
0             0     0.0     0.0     1.0  
1             3     0.0     1.

In [20]:
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X_encoded)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

(8000, 8) (2000, 8) (8000,) (2000,)


#### **Time Window Definition and Related Parameters**

Since we are working with **time series data**, we use the **`TimeSeriesProcessor` (Time Series Data Definition)** module to define the temporal window and its related parameters.  

This step allows us to specify how the data is segmented into input and output sequences, determining the **window size**, **forecasting horizon**, and other temporal configuration parameters necessary for the model.


In [21]:
wsize = 24
step = 1
n_pred = 1
features = 8


processor = TimeSeriesProcessor(window_size= wsize, step_size=1, future_prediction=False, n_pred=n_pred)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

X_train shape: (7977, 24, 8)
y_train shape: (7977, 24)
X_test shape: (1977, 24, 8)
y_test shape: (1977, 24)


In [22]:
X_train_windows = torch.tensor(X_train_windows, dtype=torch.float32)
y_train_windows = torch.tensor(y_train_windows, dtype=torch.float32)
y_train_windows = y_train_windows.unsqueeze(-1)  # to (7977, 24, 1)



#### 🤖 **Define Model**

In this example, we select the **TransformersAnomalyDetection** model from the **time series** package, choosing **`Informer`** as the model to be used.  

At this stage, we define both the **model-specific parameters** and the **training configuration**, such as the number of layers, attention heads, learning rate, batch size, and other relevant settings for model initialization and training.

In [26]:
batch_size = 16
seq_len = 24
pred_len = 24
label_len = 24
input_dim = features  # tanto encoder como decoder
d_model = 64
train_epochs = 10

# Simulamos series de entrada

# Instanciamos el modelo Transformer
kwargs = {
    "algorithm_": "informer",
    "label_parser": None, 
    "enc_in": input_dim,             # Number of input variables for the encoder
    "dec_in": input_dim,             # Number of input variables for the decoder
    "c_out": input_dim,              # Output dimension (change if needed)
    "seq_len": seq_len,              # Input sequence length
    "label_len": label_len,          # Length of the decoder input (label segment)
    "out_len": pred_len,             # Prediction length
    "factor": 5,                     # ProbSparse factor (used in ProbAttention)
    "d_model": d_model,              # Model dimension
    "n_heads": 8,                    # Number of attention heads
    "e_layers": 2,                   # Number of encoder layers
    "d_layers": 1,                   # Number of decoder layers
    "d_ff": 128,                     # Feedforward network dimension
    "dropout": 0.1,                  # Dropout rate
    "attn": "prob",                  # Attention type: 'prob' or 'full'
    "activation": "gelu",           # Activation function
    "output_attention": False,       # Whether to output attention weights
    "distil": True,                  # Whether to use distillation in the encoder
    "mix": True,                     # Whether to use mixed attention in the decoder
    "train_epochs": train_epochs,    # Number of training epochs
    "batch_size": batch_size,        # Batch size
    "lr": 1e-3                       # Learning rate
}
    

model = transformers.TransformersAnomalyDetection(**kwargs)

Train Params: {'label_parser': None, 'train_epochs': 10, 'batch_size': 16, 'lr': 0.001} 
Model Params: {'algorithm_': 'informer', 'enc_in': 8, 'dec_in': 8, 'c_out': 8, 'seq_len': 24, 'label_len': 24, 'out_len': 24, 'factor': 5, 'd_model': 64, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 128, 'dropout': 0.1, 'attn': 'prob', 'activation': 'gelu', 'output_attention': False, 'distil': True, 'mix': True}


####  **Train Model and Predict**


The model is trained on the training dataset. In this example, we perform a **basic training** to demonstrate execution, using only **10 epochs**. 
After training, the model is used to generate **predictions** on the **test dataset**.  

The **predicted values** are then compared with the **true labels** (or observed data, if available) to evaluate the **accuracy** and **robustness** of the model’s performance.
Additionally:  
- **Scores** are calculated to assess performance.  
- If true labels are available, further **metrics** can be obtained from the `metrics_module`.

In [27]:

model.fit(X_train_windows)


/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([16, 24, 8])) that is different to the input size (torch.Size([24, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([9, 24, 8])) that is different to the input size (torch.Size([24, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/10, Loss: 1.025289
Epoch 2/10, Loss: 0.996376
Epoch 3/10, Loss: 0.980781
Epoch 4/10, Loss: 0.968563
Epoch 5/10, Loss: 0.960908
Epoch 6/10, Loss: 0.957544
Epoch 7/10, Loss: 0.955777
Epoch 8/10, Loss: 0.954713
Epoch 9/10, Loss: 0.954024
Epoch 10/10, Loss: 0.953618


In [28]:
scores = model.decision_function(X_train_windows)
labels = model.predict(X_train_windows)

print("Anomaly scores:", scores.shape)
print("Predicted labels:", labels.shape)


Anomaly scores: torch.Size([7977])
Predicted labels: torch.Size([191448, 8])


In [29]:
model.evaluate(X_test_windows,y_test_windows)

Accuracy: 96.234%
F1 Score: 0.113
Recall: 0.081
Precision: 0.186


{'scores': array([0.36613294, 0.86965334, 0.9748954 , ..., 0.97224677, 8.190177  ,
        1.2645831 ], dtype=float32),
 'labels_preds': array([0, 0, 0, ..., 0, 1, 0]),
 'labels_true': tensor([0, 1, 0,  ..., 0, 1, 0])}

#### 📊 **Visualization**

For visualization, various techniques implemented in visualization_module can be used, both for **initial data exploration** and for **examining results**, depending on whether the dataset is **static** or a **time series**, and on what you want to visualize.  

If **true anomaly labels** are available, predicted anomalies can be plotted alongside them for comparison.  

For **time series scores**, we can use the class **`DataVisualizationScoresTS`** to visualize anomalies from scores. In this example, there are **three ways** to visualize these scores and the anomaly classification.  

In this case, we use the **"percentile" method** to determine anomalies, but the other methods (`"std"` and `"topk"`) are also possible.

**Parameters:**

- **Method to determine the threshold** (choose one):  
  - `"percentile"` → use the given percentile in `threshold`  
  - `"std"` → mean + `threshold` × standard deviation  
  - `"topk"` → mark the `top_k` highest scores 

In [31]:

visualizer = DataVisualizationScoresTS(scores[:4000])
fig = visualizer.visualize(method="percentile", threshold=0.95)  # percentil
fig.show()